In [ ]:
import itertools
import os
import sys

import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sk
import sklearn.preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smapi

In [ ]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))  # add parent directory to path
import samlib

# Sale price distribution
First step is to look at the target sale price for the training data set, i.e. the column we're trying to predict. 

In [ ]:
target = pd.read_csv('../data/train_target.csv')

In [ ]:
target.describe()

The sale price is in hte hundreds of thousands, so let's divide the price by 1000 to get more manageable numbers.

In [ ]:
target = target / 1000

In [ ]:
logtarget = np.log1p(target)


# Merge the training and test datasets for data preparation


In [ ]:
def read():
    """Read training and test data and return a dataframe with ['Dataset','Id'] multi-index
    """
    raw_train = pd.read_csv('../data/train_prepared_light.csv')
    raw_test = pd.read_csv('../data/test_prepared_light.csv')
    df = pd.concat([raw_train, raw_test], keys=['train', 'test'])
    df.index.names = 'Dataset', 'Id'
    return df
    
df = read()

In [ ]:
pp = samlib.Pipeline(df.copy())  
assert pp == df  # the pipeline output equals df

In [ ]:
def select_categorical_features(df):
    return df.loc[:, df.dtypes == object]

pp.append(select_categorical_features)

## Categorical features


We've got 42 categorical features. We can use the `describe` method to get some statistics:

In [ ]:
pp().shape

In [ ]:
pp().describe()

In [ ]:
pp().isnull().sum()

** Number of categories per feature **

In [ ]:
plt.figure(figsize=(12, 10))
pp().describe().loc['unique'].sort_values(ascending=False).plot(kind='barh')
plt.title('Number of categories per feature')

** Number of nulls per feature **

In [ ]:
nulls = pp().isnull().sum()

In [ ]:
plt.figure(figsize=(12, 10))
ax = nulls[nulls > 0].sort_values(ascending=False).plot(kind='barh')
plt.title('Number of nulls per feature')

But that's a lot of numbers to digest. Better get started plotting! To help with plotting, but also to improve linear regression models, we're going to standardize our data. But before that we must deal with the NaN values.
http://sebastianraschka.com/Articles/2014_about_feature_scaling.html

### Deal with NaN values 
Based on the descriptions, most of the null values are not missing values but an extra "none" category (e.g. no pool, or no alley etc). In the above graph, this is true for all features starting from 'BsmtFinType1'.

In [ ]:
def replace_bad_nulls(df):
    nulls = df.isnull().sum()
    bad_nulls_colz = nulls[nulls > 0].sort_values()['BsmtFinType1':].index
    return samlib.fillna(df, lambda x: 'none', bad_nulls_colz)

pp.append(replace_bad_nulls)

** Replace true nulls with mode **
(work in progress)

In [ ]:
pp().mode()

In [ ]:
true_nulls_colz = pp().columns[pp().isnull().sum() > 0]

In [ ]:
df = pp()
for col in true_nulls_colz:
    df = samlib.fillna(df, 'mode', col) 

In [ ]:
df.MSZoning.mode()

In [ ]:
df.isnull().sum()

## Change dtypes to `category`

In [ ]:
def object_to_category(df):
    new = df.copy()
    for column, ctype in zip(df.columns, df.dtypes):
        if ctype == object:
            new[column] = df[column].astype('category')
    return new

pp.append(object_to_category)
assert all(pp().dtypes == 'category')

## Create dummy variables

In [ ]:
pd.get_dummies(pp())